Testing data positive feedback dynamic (20%)

In [1]:
import os
import pandas as pd
import numpy as np
import random
import time
import tensorflow as tf
import math
from IPython.display import clear_output
import requests
from sklearn.metrics import average_precision_score
import json

In [2]:
def relu(x):
    return np.maximum(0,x)  

def softmax(x):
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x 

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [3]:
def lineNotifyMessage(token, msg):
    headers = {
        "Authorization": "Bearer " + token, 
        "Content-Type" : "application/x-www-form-urlencoded"
    }

    payload = {'message': msg}
    r = requests.post("https://notify-api.line.me/api/notify", headers = headers, params = payload)
    return r.status_code

In [4]:
user_following = np.load('../Data/npy/user_following_1489.npy')
#all_3374 = np.load('../Data/npy/all_2939D_img0.5.npy')
all_3374 = np.load('../Data/npy/Following_64D.npy')
user_category = np.load('../Data/npy/user_category_1489.npy')
YouTuber_category = np.load('../Data/npy/YouTuber_category_0.7.npy')
active_users = np.load('../Data/npy/active_userID_1489.npy')

In [5]:
user_test_amount = 150
yt_test_amount = 18

In [6]:
def generate_train_test(user_following,feature,user_category,item_category,active_users,user_test_amount,yt_test_amount):
    print('Generate_train_test----')
    #The shape of orignal data
    print('user_following shape ',user_following.shape)
    print('feature shape ',feature.shape)
    print('user_category shape ',user_category.shape)
    print('YouTuber_category shape ',item_category.shape)
    user_category_norm = np.zeros(user_category.shape)
    for i in range(len(user_category)):
        user_category_norm[i] = user_category[i]/np.max(user_category[i])
    print('user_category after normalized by max...')
    print('user_category_norm shape ',user_category_norm.shape)
    print('user cateogory norm',user_category_norm)
    
    #following youtuber for each user
    following_true = []
    for i in range(len(user_following)):
        each_user = []
        for j in range(len(user_following[i])):
            if user_following[i][j] == 1:
                each_user.append(j)
        following_true.append(each_user)
    #print(following_true)
    #number of followings for each user
    minlen = 10000
    maxlen = 0
    num_of_follower = []
    for i in range(len(following_true)):
        if len(following_true[i]) < minlen:
            minlen = len(following_true[i])
        if len(following_true[i]) > maxlen:
            maxlen = len(following_true[i])
        num_of_follower.append(len(following_true[i]))
    print('Min number of followings ',minlen)
    print('Max number of followings ',maxlen)
    
    num_of_users = user_following.shape[0]
    num_of_youtubers = item_category[0]
    
    """
    Spliting training and testing data
    """
    #testing user id random choice
    user_idx = [i for i in range(len(user_following))]
    random.seed(5)
    #choose test_id and sorted from small to large
    test_idx = sorted(random.sample(user_idx,user_test_amount))
    
    train_t = []
    train_f = []
    test_t = []
    test_f = []
    
    print('test_idx',test_idx)
    test_pos = -1
    for i in range(num_of_users):
        t_for_train = []
        f_for_train = []
        if i not in test_idx: #if not in test id, just append it to true or false list
            for j in range(88):
                if user_following[i][j] == 1:
                    t_for_train.append(j)
                else:
                    f_for_train.append(j)
            train_t.append(t_for_train)
            train_f.append(f_for_train)

        else: #if in test id, choose 2 true and other 
            test_pos += 1
            temp_t = []
            temp_f = []
            for j in range(88):
                if user_following[i][j] == 1:
                    temp_t.append(j)
                else:
                    temp_f.append(j)
            t_for_test = sorted(random.sample(temp_t,math.ceil(0.5*len(temp_t))))
            f_for_test  = sorted(random.sample(temp_f,yt_test_amount-len(t_for_test)))

            test_t.append(t_for_test)
            test_f.append(f_for_test)

            #other for training
            t_for_train = [item for item in temp_t if not item in t_for_test]
            f_for_train = [item for item in temp_f if not item in f_for_test]
            train_t.append(t_for_train)
            train_f.append(f_for_train)
            
    # train_t[i] 代表的是user i positive feedback
    print('The length of train_t:',len(train_t))
    print('The length of train_f:',len(train_f))
    print('The length of test_t:',len(test_t))
    print('The length of test_f:',len(test_f))
    return train_t,train_f,test_t,test_f,user_category_norm,test_idx

In [9]:
# read test_t, test_f, train_t, train_f, hopefully to get the similar score
def generate_train_test(user_following,feature,user_category,item_category,active_users,user_test_amount,yt_test_amount):
    print('Generate_train_test----')
    #The shape of orignal data
    print('user_following shape ',user_following.shape)
    print('feature shape ',feature.shape)
    print('user_category shape ',user_category.shape)
    print('YouTuber_category shape ',item_category.shape)
    user_category_norm = np.zeros(user_category.shape)
    for i in range(len(user_category)):
        user_category_norm[i] = user_category[i]/np.max(user_category[i])
    print('user_category after normalized by max...')
    print('user_category_norm shape ',user_category_norm.shape)
    print('user cateogory norm',user_category_norm)
    
    #following youtuber for each user
    following_true = []
    for i in range(len(user_following)):
        each_user = []
        for j in range(len(user_following[i])):
            if user_following[i][j] == 1:
                each_user.append(j)
        following_true.append(each_user)
    #print(following_true)
    #number of followings for each user
    minlen = 10000
    maxlen = 0
    num_of_follower = []
    for i in range(len(following_true)):
        if len(following_true[i]) < minlen:
            minlen = len(following_true[i])
        if len(following_true[i]) > maxlen:
            maxlen = len(following_true[i])
        num_of_follower.append(len(following_true[i]))
    print('Min number of followings ',minlen)
    print('Max number of followings ',maxlen)
    
    num_of_users = user_following.shape[0]
    num_of_youtubers = item_category[0]
    
    """
    Spliting training and testing data
    """
    #testing user id random choice
    user_idx = [i for i in range(len(user_following))]
    random.seed(5)
    #choose test_id and sorted from small to large
    test_idx = sorted(random.sample(user_idx,user_test_amount))
    print('test_idx',test_idx)
    
    data_split_path = 'D:/ChilliHsu/Data/Result_Matrix/Dims200_Text300/'
    with open(data_split_path+'test_t.json') as json_file:
        test_t = json.load(json_file)
    with open(data_split_path+'test_f.json') as json_file:
        test_f = json.load(json_file)
    with open(data_split_path+'train_t.json') as json_file:
        train_t = json.load(json_file)
    with open(data_split_path+'train_f.json') as json_file:
        train_f = json.load(json_file)
            
    # train_t[i] 代表的是user i positive feedback
    print('The length of train_t:',len(train_t))
    print('The length of train_f:',len(train_f))
    print('The length of test_t:',len(test_t))
    print('The length of test_f:',len(test_f))
    return train_t,train_f,test_t,test_f,user_category_norm,test_idx

In [51]:
# don't leave half positive for train, just put all testing user data into testing 
def generate_train_test(user_following,feature,user_category,item_category,active_users,user_test_amount,yt_test_amount):
    print('Generate_train_test----')
    #The shape of orignal data
    print('user_following shape ',user_following.shape)
    print('feature shape ',feature.shape)
    print('user_category shape ',user_category.shape)
    print('YouTuber_category shape ',item_category.shape)
    user_category_norm = np.zeros(user_category.shape)
    for i in range(len(user_category)):
        user_category_norm[i] = user_category[i]/np.max(user_category[i])
    print('user_category after normalized by max...')
    print('user_category_norm shape ',user_category_norm.shape)
    print('user cateogory norm',user_category_norm)
    
    #following youtuber for each user
    following_true = []
    for i in range(len(user_following)):
        each_user = []
        for j in range(len(user_following[i])):
            if user_following[i][j] == 1:
                each_user.append(j)
        following_true.append(each_user)
    #print(following_true)
    #number of followings for each user
    minlen = 10000
    maxlen = 0
    num_of_follower = []
    for i in range(len(following_true)):
        if len(following_true[i]) < minlen:
            minlen = len(following_true[i])
        if len(following_true[i]) > maxlen:
            maxlen = len(following_true[i])
        num_of_follower.append(len(following_true[i]))
    print('Min number of followings ',minlen)
    print('Max number of followings ',maxlen)
    
    num_of_users = user_following.shape[0]
    num_of_youtubers = item_category[0]
    
    """
    Spliting training and testing data
    """
    #testing user id random choice
    user_idx = [i for i in range(len(user_following))]
    random.seed(33)
    #choose test_id and sorted from small to large
    test_idx = sorted(random.sample(user_idx,user_test_amount))
    
    train_t = []
    train_f = []
    test_t = []
    test_f = []
    
    print('test_idx',test_idx)
    test_pos = -1
    for i in range(num_of_users):
        t_for_train = []
        f_for_train = []
        if i not in test_idx: #if not in test id, just append it to true or false list
            for j in range(88):
                if user_following[i][j] == 1:
                    t_for_train.append(j)
                else:
                    f_for_train.append(j)
            train_t.append(t_for_train)
            train_f.append(f_for_train)

        else: #if in test id, choose 2 true and other 
            test_pos += 1
            temp_t = []
            temp_f = []
            for j in range(88):
                if user_following[i][j] == 1:
                    temp_t.append(j)
                else:
                    temp_f.append(j)
            t_for_test = random.sample(temp_t,math.ceil(len(temp_t)))
            f_for_test  = random.sample(temp_f,yt_test_amount-len(t_for_test))

            test_t.append(t_for_test)
            test_f.append(f_for_test)

            #other for training
            t_for_train = [item for item in temp_t if not item in t_for_test]
            f_for_train = [item for item in temp_f if not item in f_for_test]
            train_t.append(t_for_train)
            train_f.append(f_for_train)
            
    # train_t[i] 代表的是user i positive feedback
    print('The length of train_t:',len(train_t))
    print('The length of train_f:',len(train_f))
    print('The length of test_t:',len(test_t))
    print('The length of test_f:',len(test_f))
    return train_t,train_f,test_t,test_f,user_category_norm,test_idx

In [7]:
train_t,train_f,test_t,test_f,user_category_norm,test_idx = generate_train_test(user_following,all_3374,user_category,YouTuber_category,active_users,user_test_amount,yt_test_amount)

Generate_train_test----
user_following shape  (1489, 88)
feature shape  (88, 64)
user_category shape  (1489, 17)
YouTuber_category shape  (88, 17)
user_category after normalized by max...
user_category_norm shape  (1489, 17)
user cateogory norm [[0.         1.         0.         ... 0.05714286 0.         0.        ]
 [0.24390244 0.         0.02439024 ... 0.         0.         0.09756098]
 [0.04210526 0.04210526 0.05263158 ... 0.02105263 0.         0.02105263]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.01408451 0.01408451 0.04225352 ... 0.02816901 0.         0.02816901]
 [0.03703704 0.22222222 0.14814815 ... 0.         0.         0.        ]]
Min number of followings  5
Max number of followings  34
test_idx [3, 6, 10, 18, 26, 37, 44, 46, 59, 65, 67, 75, 95, 99, 106, 114, 116, 133, 135, 147, 160, 165, 186, 188, 208, 221, 231, 243, 259, 270, 284, 298, 303, 304, 321, 326, 330, 339, 340, 360, 363, 370, 372, 378, 402, 403, 407, 419, 426, 428, 441, 443, 

In [8]:
test_t

[[24, 28, 29, 79],
 [11, 22, 44, 74, 82],
 [3, 11, 22, 40, 54, 63],
 [28, 43, 45, 67, 76],
 [17, 24, 57, 67, 70, 74],
 [10, 25, 40, 54, 67, 74],
 [12, 32, 33, 43, 67, 71, 84, 86],
 [8, 24, 29, 41, 63, 70],
 [24, 60, 63, 71],
 [24, 25, 28, 43, 45, 67, 75, 82],
 [3, 10, 21, 25, 40, 45, 49, 75, 80, 82],
 [10, 11, 25, 40, 67, 82],
 [3, 10, 37, 45, 49, 54, 76, 78, 79, 82],
 [49, 54, 67, 71],
 [25, 33, 45, 60, 71, 82],
 [25, 36, 71, 80],
 [3, 8, 14, 28, 29, 54, 63],
 [54, 74, 79],
 [25, 67, 74],
 [14, 40, 43, 54, 76, 79],
 [14, 25, 59, 63, 74, 82],
 [3, 20, 74],
 [11, 38, 40, 74],
 [0, 21, 32, 35, 60, 72, 74, 76, 82],
 [8, 14, 24, 28, 67, 79],
 [14, 78, 79],
 [40, 49, 75],
 [7, 11, 40, 67],
 [41, 43, 57, 63, 79],
 [33, 45, 76],
 [25, 44, 71, 74],
 [8, 28, 43, 67],
 [3, 43, 57, 79],
 [11, 12, 24, 25, 26, 27, 33, 43, 82, 83, 84],
 [24, 43, 63, 79],
 [14, 67, 74],
 [11, 28, 79],
 [14, 24, 29, 40, 43, 45, 54, 82],
 [29, 76, 79],
 [7, 10, 37, 43, 54, 67, 71, 76],
 [11, 38, 71, 74],
 [17, 27, 43, 

In [9]:
#average num of following for training user
total_train = 0
for t in train_t:
    total_train += len(t)
avg = total_train/len(user_following)
print('training',avg)
#average num of following for testing user
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test/user_test_amount
print('testing',avg)

training 9.758226997985226
testing 5.466666666666667


In [10]:
all_auxilary = [i for i in range(88)]

# Recommendation  Module

In [11]:
n = len(user_following)
m = 88  
k = 64
#l = all_3374.shape[1]

In [20]:
def training(save_name): 
    token = 'lk9WjNIjAuMTd94qdDC1oDszn38qjfCY9PiD3nvrot0'
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    loss_acc_list = []
    t0=time.time()
    
    all_trainable_vars = tf.reduce_sum([tf.reduce_prod(v.shape) for v in tf.trainable_variables()])
    print('trainable_vars:',sess.run(all_trainable_vars))
    
    train_yes_id=[] 
    for q in range(5):
        print('Iteraction:',q)
        train_auc=0
        total_loss=0
        xuij_auc=0
        length = 0
        for z in range(n):
            """
            yes 用來存放選擇到的YouTuber feature (for auxilary)
            yesr 用來存放user對該YouTuber的喜好程度(user_category 跟 YouTuber_category的相似性)
            r_3 用來存放user 對該YouTuber種類的偏好(取max)
            """
            yes=[]
            yesr=[]
        
            """
            only choose positive 
            """
            #sample=random.sample(train_t[z],len(train_t[z])) #選全部的Positive
            """
            choose all YouTuber 
            """
            sample= all_auxilary #選全部的Positive
        
            #sample=random.sample(train_t[z]+train_f[z],len(train_t[z])+len(train_f[z]))
        
            #user degree of category favor 
            r_3=np.zeros(len(sample)) 
         
            for b in range(len(sample)):
                yes.append(all_3374[sample[b]])
                yesr.append(YouTuber_category[sample[b]]*user_category_norm[z])
        
            for b in range(len(yesr)):
                r_3[b]=max(yesr[b])
            #print('r_3:',r_3)
        
            yes=np.array(yes)
        
            #取positive 
            train_t_sample = random.sample(train_t[z],len(train_t[z]))
            #print('number of positive feedback', len(train_t_sample))
        
            train_f_sample = random.sample(train_f[z],5)
            for ta in train_t_sample:
                pos = sample.index(ta)
                
                image_1=np.expand_dims(all_3374[ta],0) #(1,2048)
                #train_f_sample = random.sample(train_f[z],10)
                
                #print('Train_f_sample:', train_f_sample)
                
                for b in train_f_sample:
                    image_2=np.expand_dims(all_3374[b],0) #(1,2048)
                    
                    _last_be_relu,_norm_par,_a_list,r3,_auc, _loss,_=sess.run([last_be_relu,norm_par,a_list_smooth,a_list_soft,auc,loss,train_op], feed_dict={user: [z],
                                        i: [ta], j: [b], xf: yes , l_id:sample, l_id_len:[len(sample)],positive_id:train_t[z],positive_len:[len(train_t[z])],r:r_3,
                                        image_i:image_1,image_j:image_2})
                    #print(_a_list)
                    #print(r3)
                    train_auc+=_auc
                    total_loss+=_loss
                    length += 1
        U, Y, A, E, Au, Ay, Aa, Av,B =sess.run([user_latent, item_latent, aux_item, embedding, Wu, Wy, Wa, Wv,Beta])
        msg = test_result(U, Y, A, E,Au, Ay, Aa, Av,B,save_name+str(q))
        lineNotifyMessage(token, msg)
        np.savez('../Data/grid_search_weight/Our/'+save_name+str(q)+'.npz',U=U, Y=Y, A=A, E=E, Wu=Au, Wy=Ay, Wa=Aa, Wv=Av,B=B)
        #print('a_list:',_a_list)
        #print('a_list_soft:',r3)
        print("total_loss:-----------------", total_loss/length)
        print("train_auc:-------------------", train_auc/length)
        loss_acc_list.append([total_loss/length,train_auc/length,time.time()-t0])
        print('time:',time.time()-t0,' sec')
        print('Current time:',time.ctime())
    print('Total cost ',time.time()-t0,' sec')   
    U, Y, A, E, Au, Ay, Aa, Av,B =sess.run([user_latent, item_latent, aux_item, embedding, Wu, Wy, Wa, Wv,Beta])
    np.savez('../Data/grid_search_weight/Our/'+save_name+'.npz', 
                        U=U, Y=Y, A=A, E=E, Wu=Au, Wy=Ay, Wa=Aa, Wv=Av,B=B)
    lineNotifyMessage(token, '../Data/grid_search_weight/Our/'+save_name+'.npz')
    #np.savez('../Data/npy/mask_feature/result300/'+save_name+'.npz', 
    #                    U=U, Y=Y, A=A, E=E, Wu=Au, Wy=Ay, Wa=Aa, Wv=Av,B=B)
    
    return U, Y, A, E, Au, Ay, Aa, Av,B

In [13]:
test_amount = user_test_amount
def getScoreMatrix(RS):
    #取出test的資料
    testRS = np.zeros((test_amount,yt_test_amount)) #shape 150*18
    target = np.zeros((test_amount,yt_test_amount))
    print(testRS.shape)
    #test_t 是true的
    #test_f 是false的

    for z in range(test_amount):
        user_id = test_idx[z]
        #positive target YouTuber list
        youtube_t = test_t[z] 
        #not target YouTuber list
        youtube_f = test_f[z]
        #print(user_id)
        #print(youtube_t)
        #print(user_following[user_id])
        #前兩個放target的RS
        for i in range(len(youtube_t)):
            testRS[z][i] = RS[z][youtube_t[i]]
            target[z][i] = user_following[user_id][youtube_t[i]]
        for i in range(len(youtube_f)):
            testRS[z][i+len(youtube_t)] = RS[z][youtube_f[i]]
            target[z][i+len(youtube_t)] = user_following[user_id][youtube_f[i]]
        #print('testRS for user',user_id,testRS[z],youtube_t+youtube_f)
        
    sumtarget = 0
    for i in range(len(target)):
        #print(np.sum(target[i]))
        sumtarget += np.sum(target[i])
    print('num of positive data in testing:',sumtarget)
    print('total testing data:',test_amount*yt_test_amount)
    
    return target, testRS,sumtarget

def topN(sortlist,n):
    topList = []
    for i in range(n):
        topList.append(sortlist.index(max(sortlist)))
        #print(max(sortlist))
        #print(sortlist.index(max(sortlist)))
        sortlist[sortlist.index(max(sortlist))] = -1000000000
    return topList

def F1_score(prec,rec):
    f1 = (2*prec*rec)/(prec+rec)
    return f1
def getTOP1(target,testRS,sumtarget):
    print('top1')
    correct = 0
    for i in range(len(testRS)):
        top_0 = topN(list(testRS[i]),1) #取一個
        
        #print(top_0)
        if top_0[0] < int(np.sum(target[i])):
            correct += 1
    top1_prec = correct/len(testRS)
    top1_recall = correct/(sumtarget)
    print('prec ',top1_prec,'recall ',top1_recall)
    #f1 score
    print('F1_score:',F1_score(top1_prec,top1_recall))
    return top1_prec,top1_recall,F1_score(top1_prec,top1_recall)
def getTOP3(target,testRS,sumtarget):
    print('top3')
    correct = 0
    for i in range(len(testRS)):
        top_3 = topN(list(testRS[i]),3) #取一個
        
        #print(top_3)
        for j in range(len(top_3)):
            if top_3[j] < int(np.sum(target[i])):
                correct += 1
    top3_prec = correct/(len(testRS)*3)
    top3_recall = correct/(sumtarget)
    print('prec ',top3_prec,'recall ',top3_recall)
    #f1 score
    print('F1_score:',F1_score(top3_prec,top3_recall))
    return top3_prec,top3_recall,F1_score(top3_prec,top3_recall)
def getTOP5(target,testRS,sumtarget):
    print('top5')
    correct = 0
    for i in range(len(testRS)):
        top_5 = topN(list(testRS[i]),5) #取一個
       
        #print(top_5)
        for j in range(len(top_5)):
            if top_5[j] < int(np.sum(target[i])):
                correct += 1
    top5_prec = correct/(len(testRS)*5)
    top5_recall = correct/(sumtarget)
    print('prec ',top5_prec,'recall ',top5_recall)
    #f1 score
    print('F1_score:',F1_score(top5_prec,top5_recall))
    return top5_prec,top5_recall,F1_score(top5_prec,top5_recall)

In [14]:
"""
NDCG
https://daiwk.github.io/posts/nlp-ndcg.html
"""
# pre_list
"""
test_amount = 150
yt_test_amount = 18
"""
def NDCG(target,testRS): #target是真正的喜好
    all_sort = []
    num_ndcg = 10
    pre_matrix = np.zeros(shape=(test_amount,yt_test_amount)) #(150,18)
    for i in range(test_amount): #user amount = 150
        top_n = topN(list(testRS[i]),num_ndcg) #取10個
        #print(top_n)
        all_sort.append(topN(list(testRS[i]),len(testRS[i])))
        #print('all_sort',topN(list(testRS[i]),len(testRS[i])))
        for j in range(len(top_n)):
            pre_matrix[i][top_n[j]] = 1

    #Ideal DCG，理想状况下的DCG。也就是说，相关性完全由高到低排序时算出的DCG：
    def IDCG(ideal_list): #ideal_list example = [1,1,1,1,1,0,0,....]
        idcg=0
        #print('ideal',ideal_list)
        for i in range(len(ideal_list)):
            #print((2**true_list[i]-1),math.log2(i+2))
            idcg+= (2**ideal_list[i]-1)/ math.log2(i+2)
        #print('idcg',idcg)
        return idcg
    def DCG(prec_list): #找出前n名的[1,1,1,0,...]
        dcg=0
        #print('prec',prec_list)
        for i in range(len(prec_list)):
            dcg+= (2**prec_list[i]-1)/ math.log2(i+2)
        #print('dcg',dcg)
        return dcg
    total_ndcg = 0
    
    for m in range(test_amount): # the number of testing users
        #print('all-sort',all_sort[m][:num_ndcg])
        idcg = IDCG(target[m][:num_ndcg])
        pre_list = []
        least_pre_list = []
        for s in all_sort[m][:num_ndcg]:
            #print(m,s,target[m][s])
            pre_list.append(target[m][s]) #把prec_list 的 score加進去
        for s in all_sort[m][num_ndcg:]:
            #print(s)
            #print(target[m][s])
            least_pre_list.append(target[m][s]) #把prec_list 的 score加進去
        dcg = DCG(pre_list)
        ndcg = dcg/idcg
        #print(ndcg)
        total_ndcg += ndcg
    avg_ndcg = total_ndcg/test_amount
    print('NDCG:',avg_ndcg)
    return pre_matrix,avg_ndcg

# MAP
"""
>>> y_true = np.array([0, 0, 1, 1])
>>> y_scores = np.array([0.1, 0.4, 0.35, 0.8])
>>> average_precision_score(y_true, y_scores)
"""
def MAP(target,testRS):
    print('target:',target)
    print('testRS:',testRS)
    total_prec = 0
    for u in range(test_amount):
        y_true = target[u]
        y_scores = testRS[u]
        total_prec+=average_precision_score(y_true, y_scores)
    Map_value = total_prec/test_amount
    print('MAP',Map_value)
    return Map_value

In [15]:
def testing(U, Y, A, E,Au, Ay, Aa, Av,B):
    test_amount = 150
    yt_test_amount = 18
    result=np.zeros((test_amount,88))
    RS=np.zeros((test_amount,88))
    #test_idx --> Test 的 index
    print(test_idx)
    sum_alpha = 0
    for s in range(test_amount):
        #print(s,test_idx[s])

        yes=[]
        sample = [i for i in range(88)]
        alpha=np.zeros([len(sample)])

        for a in range(len(sample)):
            r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
            alpha_a = np.dot(Au[test_idx[s]][sample[a]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[test_idx[s]][sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]][sample[a]],
                    np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]][sample[a]],np.dot(E,np.expand_dims(all_3374[sample[a]],0).T))
            """
            relu part ...
            """
            alpha[a]=np.sum((relu(alpha_a)))*r
            """
            tanh part ...
            """
            #alpha[a]=np.sum((np.tanh(alpha_a)))*r
            
            
        mul=np.zeros((1,64))
        #print('alpha--------',alpha)
        #print('add alpha------------',np.add(alpha,0.000000001))
        added_alpha = np.add(alpha,0.0000000001)
        norm_alpha = added_alpha/np.sum(added_alpha)
        #print('alpha-----------',alpha)
        #print('norm alpha--------------',norm_alpha)
        sum_alpha += np.sum(alpha)
        for i in range(len(sample)):
            mul+=norm_alpha[i]*A[sample[i]] #attention alpha*Ai part
        new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    
        
        
        for k in range(88):
            result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
            RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]],np.dot(E, all_3374[k].T))
            
    return RS

In [16]:
def test_result(U, Y, A, E,Wu, Wy, Wa, Wv,B,save_name):
    RS = testing(U, Y, A, E,Wu, Wy, Wa, Wv,B)
    target,testRS,sumtarget = getScoreMatrix(RS)
    prec_1,recall_1,f1_1 = getTOP1(target,testRS,sumtarget)
    prec_3,recall_3,f1_3 = getTOP3(target,testRS,sumtarget)
    prec_5,recall_5,f1_5 = getTOP5(target,testRS,sumtarget)
    pre_matrix,avg_ndcg = NDCG(target,testRS)
    Map_value = MAP(target,testRS)
    msg = save_name+'\nNDCG:'+str(round(avg_ndcg,3))+'\n'+'MAP:'+str(round(Map_value,3))+'\nTop1: prec'+str(round(prec_1,3))+' rec'+str(round(recall_1,3))+' f1'+str(round(f1_1,3))+'\nTop3: prec'+str(round(prec_3,3))+' rec'+str(round(recall_3,3))+' f1'+str(round(f1_3,3))+'\nTop5: prec'+str(round(prec_5,3))+' rec'+str(round(recall_5,3))+' f1'+str(round(f1_5,3))
    
    return msg

# Main Loop

In [19]:
all_3374.shape

(88, 2112)

In [ ]:
# two feature 
par_weights = [0.01]
beta_weights = [0.001]
Embedding_weights = [0.01]
Embedding_dims = [200]
#l = all_3374.shape[1]
try_count = [str(try_i) for try_i in range(3)]
feature_name = ['Following_64D','Image_50_2048D','Text_300D','Video_512D_6650','social_Norm15D']
token = 'lk9WjNIjAuMTd94qdDC1oDszn38qjfCY9PiD3nvrot0'
testcount = 0
finish_list = []
all_3374 = np.load('../Data/npy/'+'Following_64D'+'.npy')
for feature in feature_name[1:]:
    #added_feature = np.load('../Data/npy/'+'Image_50_2048D'+'.npy')
    #all_3374 = np.concatenate((all_3374,added_feature),axis=1)
    if 'Following' not in feature:
        added_feature = np.load('../Data/npy/'+feature+'.npy')
        all_3374 = np.concatenate((all_3374,added_feature),axis=1)
        l = all_3374.shape[1]
    else:
        l = all_3374.shape[1]
    for pary_weight in par_weights:
        for beta_weight in beta_weights:
            for Embedding_weight in Embedding_weights:
                for embedding_dims in Embedding_dims:
                    #clear_output()
                    print('Finished',finish_list)
                    finish_list.append(feature)
                    print(l)
                    print('Now Dims:',embedding_dims)
                    """
                    n: the number of users
                    m: the number of YouTubers
                    k: latent dims
                    l: feature dims
                    """
                    lineNotifyMessage(token,feature)
                    tf.reset_default_graph()

                    user = tf.placeholder(tf.int32,shape=(1,))
                    i = tf.placeholder(tf.int32, shape=(1,))
                    j = tf.placeholder(tf.int32, shape=(1,))

                    #多少個auxliary 
                    xf = tf.placeholder(tf.float32, shape=(None,l))
                    l_id = tf.placeholder(tf.int32, shape=(None,))
                    l_id_len = tf.placeholder(tf.int32,shape=(1,))
                    positive_id = tf.placeholder(tf.int32, shape=(None,))
                    positive_len = tf.placeholder(tf.int32,shape=(1,))
                    r = tf.placeholder(tf.float32,shape=(None,))


                    image_i = tf.placeholder(tf.float32, shape=(1,l))
                    image_j = tf.placeholder(tf.float32, shape=(1,l))
                    
                    
                    with tf.variable_scope("item_level"):
                        user_latent = tf.get_variable("user_latent", [n, k],
                                                              initializer=tf.random_normal_initializer(0.1,0.01,seed=3))
                        item_latent = tf.get_variable("item_latent", [m, k],
                                                              initializer=tf.random_normal_initializer(0.1,0.01,seed=3)) 
                        aux_item = tf.get_variable("aux_item", [m, k],
                                                              initializer=tf.random_normal_initializer(0.1,0.01,seed=3))
                        Wu = tf.get_variable("Wu", [n,m,k],  
                                                              initializer=tf.contrib.layers.xavier_initializer())
                        Wy = tf.get_variable("Wy", [n,m,k],   
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wa = tf.get_variable("Wa", [n,m,k],  
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wv = tf.get_variable("Wv", [n,m,embedding_dims],  
                                                            initializer=tf.contrib.layers.xavier_initializer())
                        #Wve =  tf.get_variable("Wve", [embedding_dims,l],  
                        #                                    initializer=tf.contrib.layers.xavier_initializer())

                        #每個user 對於每個YouTuber都有一個權重
                        #w1拿掉，wu
                        #hyper?

                        aux_new = tf.get_variable("aux_new", [1,k], initializer=tf.constant_initializer(0.0))
                        ########## Error part, how to get auxisize dynamically
                        ####aux_size= tf.get_variable(name='aux_size', initializer=l_id.get_shape().as_list()[-1])

                    with tf.variable_scope('feature_level'):
                        embedding = tf.get_variable("embedding", [embedding_dims,l],
                                    initializer=tf.contrib.layers.xavier_initializer())
                        Beta = tf.get_variable("beta", [n,embedding_dims],
                                    initializer=tf.random_normal_initializer(0.1,0.001,seed=10))

                    #lookup the latent factors by user and id
                    u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
                    vi = tf.nn.embedding_lookup(item_latent, i) 
                    vj = tf.nn.embedding_lookup(item_latent, j)


                    wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user)) #(m*k)
                    wy = tf.squeeze(tf.nn.embedding_lookup(Wy, user)) #(m*k)
                    wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user)) #(m*k)
                    wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user)) #(m,l)
                    beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor


                    a_list=tf.Variable([])
                    q = tf.constant(0)
                    def att_cond(q,a_list):
                        return tf.less(q,l_id_len[0])
                    def att_body(q,a_list):
                        xfi = tf.expand_dims(xf[q],0) #(1,l)
                        wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        a_list = tf.concat([a_list,[(tf.nn.relu( tf.matmul(wuui, u, transpose_b=True) +
                                tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
                                tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
                                tf.matmul(wvui,tf.matmul(embedding,xfi, transpose_b=True)))[0][0])*r[q]]],0)
                        q += 1
                        return q,  a_list

                    _, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])

                    # for while for smoothing
                    #a_list_soft=tf.nn.softmax(a_list)
                    a_list_smooth = tf.add(a_list,0.0000000001)
                    a_list_soft = tf.divide(a_list_smooth,tf.reduce_sum(a_list_smooth, 0)) #without softmax

                    norm_par = [wu,wy,wa,wv]

                    wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[-1]),0)
                    wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[-1]),0)
                    waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[-1]),0)
                    wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[-1]),0)
                    wu_be_relu = tf.matmul(wuui, u, transpose_b=True)
                    wy_be_relu = tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[-1]),0), transpose_b=True)
                    wa_be_relu = tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[-1]),0), transpose_b=True)
                    wv_be_relu = tf.matmul(wvui, tf.matmul(embedding,tf.expand_dims(xf[-1],0), transpose_b=True))
                    last_be_relu = [wu_be_relu,wy_be_relu,wa_be_relu,wv_be_relu]

                    aux_np = tf.expand_dims(tf.zeros(k),0) #dimension (1,32)
                    q = tf.constant(0)
                    def sum_att_cond(q,aux_np):
                        return tf.less(q,l_id_len[0])

                    def sum_att_body(q,aux_np):
                        #aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                        aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)]) 
                        q += 1
                        return q, aux_np

                    _,aux_np = tf.while_loop(sum_att_cond,sum_att_body,[q,aux_np])

                    """
                    for q in range(3): #取q個auxliary item
                        aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                    """

                    aux_part = tf.matmul(aux_np, vi, transpose_b=True)
                    #tf.print('aux attention:',aux_np)
                    aux_np+=u #user_latent factor + sum (alpha*auxilary)
                    aux_new=tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np


                    latent_i_part = tf.matmul(aux_new, vi, transpose_b=True)
                    feature_i_part = tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                    latent_j_part = tf.matmul(aux_new, vj, transpose_b=True)
                    feature_j_part = tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))
                    only_aux_i_part = tf.matmul(aux_np, vi, transpose_b=True)
                    only_aux_j_part = tf.matmul(aux_np, vj, transpose_b=True)

                    #矩陣中對應函數各自相乘
                    # ex: tf.matmul(thetav,(tf.matmul(embedding, image_i, transpose_b=True)))
                    xui = tf.matmul(aux_new, vi, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                    xuj = tf.matmul(aux_new, vj, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))

                    xuij = tf.subtract(xui,xuj)


                    l2_norm = tf.add_n([
                                0.01 * tf.reduce_sum(tf.multiply(u, u)),
                                0.01 * tf.reduce_sum(tf.multiply(vi, vi)),
                                0.01 * tf.reduce_sum(tf.multiply(vj, vj)),


                                0.01 * tf.reduce_sum(tf.multiply(wu, wu)),
                                pary_weight * tf.reduce_sum(tf.multiply(wy, wy)),
                                pary_weight * tf.reduce_sum(tf.multiply(wa, wa)),
                                pary_weight * tf.reduce_sum(tf.multiply(wv,wv)),

                                beta_weight * tf.reduce_sum(tf.multiply(beta,beta)),
                                Embedding_weight * tf.reduce_sum(tf.multiply(embedding,embedding)),
                    ])
                                
                        

                    loss = l2_norm -tf.log(tf.sigmoid(xuij)) # objective funtion
                    train_op = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(loss) #parameter optimize 
                    auc = tf.reduce_mean(tf.to_float(xuij > 0))
                    merge_f = ''
                    for fname in finish_list:
                        merge_f=merge_f+fname
                    save_name = merge_f+'6_18Result_lr5'
                    Ur, Yr, Ar, Er, Aur, Ayr, Aar, Avr,Br = training(save_name)
                    #np.savez(save_name+'.npz', 
                    #    U=Ur, Y=Yr, A=Ar, E=Er, Wu=Aur, Wy=Ayr, Wa=Aar, Wv=Avr,B=Br)
                    print('Finish Training:',save_name)
                    print('Start Testing:')
                    msg = test_result(Ur, Yr, Ar, Er,Aur, Ayr, Aar, Avr,Br,save_name)
                    lineNotifyMessage(token, msg)

Finished []
2112
Now Dims: 200
trainable_vars: 52191368
Iteraction: 0


In [ ]:
# two feature 
par_weights = [0.01]
beta_weights = [0.001]
Embedding_weights = [0.01]
Embedding_dims = [200]
#l = all_3374.shape[1]
try_count = [str(try_i) for try_i in range(3)]
feature_name = ['Following_64D','Image_50_2048D','Text_300D','Video_512D_6650','social_Norm15D']
token = 'lk9WjNIjAuMTd94qdDC1oDszn38qjfCY9PiD3nvrot0'
testcount = 0
finish_list = []
all_3374 = np.load('../Data/npy/'+'Following_64D'+'.npy')
for feature in feature_name:
    if 'Following' not in feature:
        added_feature = np.load('../Data/npy/'+feature+'.npy')
        all_3374 = np.concatenate((all_3374,added_feature),axis=1)
        l = all_3374.shape[1]
    else:
        l = all_3374.shape[1]
    for pary_weight in par_weights:
        for beta_weight in beta_weights:
            for Embedding_weight in Embedding_weights:
                for embedding_dims in Embedding_dims:
                    #clear_output()
                    print('Finished',finish_list)
                    finish_list.append(feature)
                    print(l)
                    print('Now Dims:',embedding_dims)
                    """
                    n: the number of users
                    m: the number of YouTubers
                    k: latent dims
                    l: feature dims
                    """
                    lineNotifyMessage(token,'One_feature '+feature)
                    tf.reset_default_graph()

                    user = tf.placeholder(tf.int32,shape=(1,))
                    i = tf.placeholder(tf.int32, shape=(1,))
                    j = tf.placeholder(tf.int32, shape=(1,))

                    #多少個auxliary 
                    xf = tf.placeholder(tf.float32, shape=(None,l))
                    l_id = tf.placeholder(tf.int32, shape=(None,))
                    l_id_len = tf.placeholder(tf.int32,shape=(1,))
                    positive_id = tf.placeholder(tf.int32, shape=(None,))
                    positive_len = tf.placeholder(tf.int32,shape=(1,))
                    r = tf.placeholder(tf.float32,shape=(None,))


                    image_i = tf.placeholder(tf.float32, shape=(1,l))
                    image_j = tf.placeholder(tf.float32, shape=(1,l))
                    
                    
                    with tf.variable_scope("item_level"):
                        user_latent = tf.get_variable("user_latent", [n, k],
                                                              initializer=tf.random_normal_initializer(0.1,0.001,seed=3))
                        item_latent = tf.get_variable("item_latent", [m, k],
                                                              initializer=tf.random_normal_initializer(0.1,0.001,seed=3)) 
                        aux_item = tf.get_variable("aux_item", [m, k],
                                                              initializer=tf.random_normal_initializer(0.1,0.001,seed=3))
                        Wu = tf.get_variable("Wu", [n,m,k],  
                                                              initializer=tf.contrib.layers.xavier_initializer())
                        Wy = tf.get_variable("Wy", [n,m,k],   
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wa = tf.get_variable("Wa", [n,m,k],  
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wv = tf.get_variable("Wv", [n,m,embedding_dims],  
                                                            initializer=tf.contrib.layers.xavier_initializer())
                        #Wve =  tf.get_variable("Wve", [embedding_dims,l],  
                        #                                    initializer=tf.contrib.layers.xavier_initializer())

                        #每個user 對於每個YouTuber都有一個權重
                        #w1拿掉，wu
                        #hyper?

                        aux_new = tf.get_variable("aux_new", [1,k], initializer=tf.constant_initializer(0.0))
                        ########## Error part, how to get auxisize dynamically
                        ####aux_size= tf.get_variable(name='aux_size', initializer=l_id.get_shape().as_list()[-1])

                    with tf.variable_scope('feature_level'):
                        embedding = tf.get_variable("embedding", [embedding_dims,l],
                                    initializer=tf.contrib.layers.xavier_initializer())
                        Beta = tf.get_variable("beta", [n,embedding_dims],
                                    initializer=tf.random_normal_initializer(0.01,0.001,seed=10))

                    #lookup the latent factors by user and id
                    u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
                    vi = tf.nn.embedding_lookup(item_latent, i) 
                    vj = tf.nn.embedding_lookup(item_latent, j)


                    wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user)) #(m*k)
                    wy = tf.squeeze(tf.nn.embedding_lookup(Wy, user)) #(m*k)
                    wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user)) #(m*k)
                    wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user)) #(m,l)
                    beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor


                    a_list=tf.Variable([])
                    q = tf.constant(0)
                    def att_cond(q,a_list):
                        return tf.less(q,l_id_len[0])
                    def att_body(q,a_list):
                        xfi = tf.expand_dims(xf[q],0) #(1,l)
                        wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        a_list = tf.concat([a_list,[(tf.nn.relu( tf.matmul(wuui, u, transpose_b=True) +
                                tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
                                tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
                                tf.matmul(wvui,tf.matmul(embedding,xfi, transpose_b=True)))[0][0])*r[q]]],0)
                        q += 1
                        return q,  a_list

                    _, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])

                    # for while for smoothing
                    #a_list_soft=tf.nn.softmax(a_list)
                    a_list_smooth = tf.add(a_list,0.0000000001)
                    a_list_soft = tf.divide(a_list_smooth,tf.reduce_sum(a_list_smooth, 0)) #without softmax

                    norm_par = [wu,wy,wa,wv]

                    wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[-1]),0)
                    wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[-1]),0)
                    waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[-1]),0)
                    wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[-1]),0)
                    wu_be_relu = tf.matmul(wuui, u, transpose_b=True)
                    wy_be_relu = tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[-1]),0), transpose_b=True)
                    wa_be_relu = tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[-1]),0), transpose_b=True)
                    wv_be_relu = tf.matmul(wvui, tf.matmul(embedding,tf.expand_dims(xf[-1],0), transpose_b=True))
                    last_be_relu = [wu_be_relu,wy_be_relu,wa_be_relu,wv_be_relu]

                    aux_np = tf.expand_dims(tf.zeros(k),0) #dimension (1,32)
                    q = tf.constant(0)
                    def sum_att_cond(q,aux_np):
                        return tf.less(q,l_id_len[0])

                    def sum_att_body(q,aux_np):
                        #aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                        aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)]) 
                        q += 1
                        return q, aux_np

                    _,aux_np = tf.while_loop(sum_att_cond,sum_att_body,[q,aux_np])

                    """
                    for q in range(3): #取q個auxliary item
                        aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                    """

                    aux_part = tf.matmul(aux_np, vi, transpose_b=True)
                    #tf.print('aux attention:',aux_np)
                    aux_np+=u #user_latent factor + sum (alpha*auxilary)
                    aux_new=tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np


                    latent_i_part = tf.matmul(aux_new, vi, transpose_b=True)
                    feature_i_part = tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                    latent_j_part = tf.matmul(aux_new, vj, transpose_b=True)
                    feature_j_part = tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))
                    only_aux_i_part = tf.matmul(aux_np, vi, transpose_b=True)
                    only_aux_j_part = tf.matmul(aux_np, vj, transpose_b=True)

                    #矩陣中對應函數各自相乘
                    # ex: tf.matmul(thetav,(tf.matmul(embedding, image_i, transpose_b=True)))
                    xui = tf.matmul(aux_new, vi, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                    xuj = tf.matmul(aux_new, vj, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))

                    xuij = tf.subtract(xui,xuj)


                    l2_norm = tf.add_n([
                                0.01 * tf.reduce_sum(tf.multiply(u, u)),
                                0.01 * tf.reduce_sum(tf.multiply(vi, vi)),
                                0.01 * tf.reduce_sum(tf.multiply(vj, vj)),


                                0.01 * tf.reduce_sum(tf.multiply(wu, wu)),
                                pary_weight * tf.reduce_sum(tf.multiply(wy, wy)),
                                pary_weight * tf.reduce_sum(tf.multiply(wa, wa)),
                                pary_weight * tf.reduce_sum(tf.multiply(wv,wv)),

                                beta_weight * tf.reduce_sum(tf.multiply(beta,beta)),
                                Embedding_weight * tf.reduce_sum(tf.multiply(embedding,embedding)),
                    ])
                                
                        

                    loss = l2_norm -tf.log(tf.sigmoid(xuij)) # objective funtion
                    train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss) #parameter optimize 
                    auc = tf.reduce_mean(tf.to_float(xuij > 0))
                    merge_f = ''
                    for fname in finish_list:
                        merge_f=merge_f+fname
                    save_name = merge_f+'_18Result_lr4'
                    Ur, Yr, Ar, Er, Aur, Ayr, Aar, Avr,Br = training(save_name)
                    #np.savez(save_name+'.npz', 
                    #    U=Ur, Y=Yr, A=Ar, E=Er, Wu=Aur, Wy=Ayr, Wa=Aar, Wv=Avr,B=Br)
                    print('Finish Training:',save_name)
                    print('Start Testing:')
                    msg = test_result(Ur, Yr, Ar, Er,Aur, Ayr, Aar, Avr,Br,save_name)
                    lineNotifyMessage(token, msg)

In [65]:
print('Start Testing:')
msg = test_result(Ur, Yr, Ar, Er,Aur, Ayr, Aar, Avr,Br,save_name)
lineNotifyMessage(token, msg)

Start Testing:
[3, 6, 10, 18, 26, 37, 44, 46, 59, 65, 67, 75, 95, 99, 106, 114, 116, 133, 135, 147, 160, 165, 186, 188, 208, 221, 231, 243, 259, 270, 284, 298, 303, 304, 321, 326, 330, 339, 340, 360, 363, 370, 372, 378, 402, 403, 407, 419, 426, 428, 441, 443, 486, 503, 504, 510, 512, 513, 523, 524, 540, 564, 572, 592, 605, 611, 617, 626, 627, 633, 634, 639, 642, 646, 648, 679, 692, 696, 697, 704, 705, 726, 727, 732, 734, 739, 742, 749, 752, 761, 770, 779, 784, 797, 827, 831, 835, 849, 857, 863, 886, 911, 927, 933, 946, 947, 953, 960, 967, 984, 985, 990, 1049, 1050, 1074, 1085, 1092, 1104, 1113, 1116, 1124, 1175, 1184, 1200, 1207, 1216, 1220, 1230, 1235, 1250, 1264, 1265, 1275, 1277, 1281, 1283, 1307, 1329, 1333, 1335, 1388, 1404, 1411, 1414, 1426, 1438, 1443, 1449, 1474, 1476]
(150, 88)
num of positive data in testing: 1566.0
total testing data: 13200
top1
prec  0.6666666666666666 recall  0.06385696040868455
F1_score: 0.11655011655011654
top3
prec  0.6288888888888889 recall  0.18071519

200

In [ ]:
par_weights = [0.01]
beta_weights = [0.001]
Embedding_weights = [0.01]
Embedding_dims = [200]
all_3374 = np.load('../Data/npy/mask_feature/no_300image.npy')
l = all_3374.shape[1]
only_features = ['no_300image']
try_count = [str(try_i) for try_i in range(10)]

testcount = 0
finish_list = []
for only_feature in only_features:
    all_3374 = np.load('../Data/npy/mask_feature/'+only_feature+'.npy')
    l = all_3374.shape[1]
    for pary_weight in par_weights:
        for beta_weight in beta_weights:
            for Embedding_weight in Embedding_weights:
                for embedding_dims in Embedding_dims:
                    clear_output()
                    print('Finished',finish_list)
                    finish_list.append(only_feature)
                    print(l)
                    print('Now Dims:',embedding_dims)
                    """
                    n: the number of users
                    m: the number of YouTubers
                    k: latent dims
                    l: feature dims
                    """
                    tf.reset_default_graph()

                    user = tf.placeholder(tf.int32,shape=(1,))
                    i = tf.placeholder(tf.int32, shape=(1,))
                    j = tf.placeholder(tf.int32, shape=(1,))

                    #多少個auxliary 
                    xf = tf.placeholder(tf.float32, shape=(None,l))
                    l_id = tf.placeholder(tf.int32, shape=(None,))
                    l_id_len = tf.placeholder(tf.int32,shape=(1,))
                    positive_id = tf.placeholder(tf.int32, shape=(None,))
                    positive_len = tf.placeholder(tf.int32,shape=(1,))
                    r = tf.placeholder(tf.float32,shape=(None,))


                    image_i = tf.placeholder(tf.float32, shape=(1,l))
                    image_j = tf.placeholder(tf.float32, shape=(1,l))

                    with tf.variable_scope("item_level"):
                        user_latent = tf.get_variable("user_latent", [n, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=4))
                        item_latent = tf.get_variable("item_latent", [m, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=6)) 
                        aux_item = tf.get_variable("aux_item", [m, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=8))
                        Wu = tf.get_variable("Wu", [n,m,k],  
                                                              initializer=tf.contrib.layers.xavier_initializer())
                        Wy = tf.get_variable("Wy", [n,m,k],   
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wa = tf.get_variable("Wa", [n,m,k],  
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wv = tf.get_variable("Wv", [n,m,embedding_dims],  
                                                            initializer=tf.contrib.layers.xavier_initializer())
                        #Wve =  tf.get_variable("Wve", [embedding_dims,l],  
                        #                                    initializer=tf.contrib.layers.xavier_initializer())

                        #每個user 對於每個YouTuber都有一個權重
                        #w1拿掉，wu
                        #hyper?

                        aux_new = tf.get_variable("aux_new", [1,k], initializer=tf.constant_initializer(0.0))
                        ########## Error part, how to get auxisize dynamically
                        ####aux_size= tf.get_variable(name='aux_size', initializer=l_id.get_shape().as_list()[-1])

                    with tf.variable_scope('feature_level'):
                        embedding = tf.get_variable("embedding", [embedding_dims,l],
                                    initializer=tf.contrib.layers.xavier_initializer())
                        Beta = tf.get_variable("beta", [n,embedding_dims],
                                    initializer=tf.random_normal_initializer(0.01,0.001,seed=10))

                    #lookup the latent factors by user and id
                    u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
                    vi = tf.nn.embedding_lookup(item_latent, i) 
                    vj = tf.nn.embedding_lookup(item_latent, j)


                    wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user)) #(m*k)
                    wy = tf.squeeze(tf.nn.embedding_lookup(Wy, user)) #(m*k)
                    wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user)) #(m*k)
                    wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user)) #(m,l)
                    beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor


                    a_list=tf.Variable([])
                    q = tf.constant(0)
                    def att_cond(q,a_list):
                        return tf.less(q,l_id_len[0])
                    def att_body(q,a_list):
                        xfi = tf.expand_dims(xf[q],0) #(1,l)
                        wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        a_list = tf.concat([a_list,[(tf.nn.relu( tf.matmul(wuui, u, transpose_b=True) +
                                tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
                                tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
                                tf.matmul(wvui,tf.matmul(embedding,xfi, transpose_b=True)))[0][0])*r[q]]],0)
                        q += 1
                        return q,  a_list

                    _, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])

                    # for while for smoothing
                    #a_list_soft=tf.nn.softmax(a_list)
                    a_list_smooth = tf.add(a_list,0.0000000001)
                    a_list_soft = tf.divide(a_list_smooth,tf.reduce_sum(a_list_smooth, 0)) #without softmax

                    norm_par = [wu,wy,wa,wv]

                    wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[-1]),0)
                    wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[-1]),0)
                    waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[-1]),0)
                    wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[-1]),0)
                    wu_be_relu = tf.matmul(wuui, u, transpose_b=True)
                    wy_be_relu = tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[-1]),0), transpose_b=True)
                    wa_be_relu = tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[-1]),0), transpose_b=True)
                    wv_be_relu = tf.matmul(wvui, tf.matmul(embedding,tf.expand_dims(xf[-1],0), transpose_b=True))
                    last_be_relu = [wu_be_relu,wy_be_relu,wa_be_relu,wv_be_relu]

                    aux_np = tf.expand_dims(tf.zeros(k),0) #dimension (1,32)
                    q = tf.constant(0)
                    def sum_att_cond(q,aux_np):
                        return tf.less(q,l_id_len[0])

                    def sum_att_body(q,aux_np):
                        #aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                        aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)]) 
                        q += 1
                        return q, aux_np

                    _,aux_np = tf.while_loop(sum_att_cond,sum_att_body,[q,aux_np])

                    """
                    for q in range(3): #取q個auxliary item
                        aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                    """

                    aux_part = tf.matmul(aux_np, vi, transpose_b=True)
                    #tf.print('aux attention:',aux_np)
                    aux_np+=u #user_latent factor + sum (alpha*auxilary)
                    aux_new=tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np


                    latent_i_part = tf.matmul(aux_new, vi, transpose_b=True)
                    feature_i_part = tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                    latent_j_part = tf.matmul(aux_new, vj, transpose_b=True)
                    feature_j_part = tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))
                    only_aux_i_part = tf.matmul(aux_np, vi, transpose_b=True)
                    only_aux_j_part = tf.matmul(aux_np, vj, transpose_b=True)

                    #矩陣中對應函數各自相乘
                    # ex: tf.matmul(thetav,(tf.matmul(embedding, image_i, transpose_b=True)))
                    xui = tf.matmul(aux_new, vi, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                    xuj = tf.matmul(aux_new, vj, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))

                    xuij = tf.subtract(xui,xuj)


                    l2_norm = tf.add_n([
                                0.0001 * tf.reduce_sum(tf.multiply(u, u)),
                                0.0001 * tf.reduce_sum(tf.multiply(vi, vi)),
                                0.0001 * tf.reduce_sum(tf.multiply(vj, vj)),


                                0.01 * tf.reduce_sum(tf.multiply(wu, wu)),
                                pary_weight * tf.reduce_sum(tf.multiply(wy, wy)),
                                pary_weight * tf.reduce_sum(tf.multiply(wa, wa)),
                                pary_weight * tf.reduce_sum(tf.multiply(wv,wv)),

                                beta_weight * tf.reduce_sum(tf.multiply(beta,beta)),
                                Embedding_weight * tf.reduce_sum(tf.multiply(embedding,embedding)),
                    ])
                                
                        

                    loss = l2_norm -tf.log(tf.sigmoid(xuij)) # objective funtion
                    train_op = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(loss) #parameter optimize 
                    auc = tf.reduce_mean(tf.to_float(xuij > 0))
                    
                    Ur, Yr, Ar, Er, Aur, Ayr, Aar, Avr,Br = training(only_feature+'9_E'+str(embedding_dims))
                    print('Finish dims:,',embedding_dims)

In [ ]:
a = np.load('../Data/npy/Image_50_2048D.npy')
b = np.load('../Data/npy/Video_512D_6650.npy')
#c = np.load('../Data/npy/social_Norm15D.npy')

all_3374 = np.concatenate((a, b), axis=1)
#all_3374=np.concatenate((all_3374, c), axis=1)
l =all_3374.shape[1]
print('all 3374 shape',all_3374.shape)

In [ ]:
#finish list:[image+video,image+text,video+text,social_text,social_image,social_video, following_video, following_social,following_image,following_text]
#finish list:[following_image_text,following_image_video,following_image_social,social_image_video,social_image_text]
par_weights = [0.01]
beta_weights = [0.001]
Embedding_weights = [0.01]
#Embedding_dims = [150,160,170,180,190,210,220,230,240,250]
Embedding_dims = [200]
single_features = ['Video','Text','Social','Following','Image']
testcount = 0
finish_list = []
for paru_weight in par_weights:
    for pary_weight in par_weights:
        for beta_weight in beta_weights:
            for Embedding_weight in Embedding_weights:
                for single_feature  in single_features:
                    if 'Video' in single_feature:
                        all_3374 = np.load('../Data/npy/Video_512D_6650.npy')
                    elif 'Text' in single_feature:
                        all_3374 = np.load('../Data/npy/Text_300D.npy')
                    elif 'Social' in single_feature:
                        all_3374 = np.load('../Data/npy/social_15D.npy')
                    elif 'Following' in single_feature:
                        all_3374 = np.load('../Data/npy/Following_64D.npy')
                    elif 'Image' in single_feature:
                        all_3374 = np.load('../Data/npy/Image_50_2048D.npy')
                        
                    l = all_3374.shape[1]
                    
                    embedding_dims = 200
                    clear_output()
                    print('Finished Dims',finish_list)
                    finish_list.append(embedding_dims)
                    print('Now feature:',single_feature,l)
                    """
                    n: the number of users
                    m: the number of YouTubers
                    
                    k: latent dims
                    l: feature dims
                    """
                    tf.reset_default_graph()

                    user = tf.placeholder(tf.int32,shape=(1,))
                    i = tf.placeholder(tf.int32, shape=(1,))
                    j = tf.placeholder(tf.int32, shape=(1,))

                    #多少個auxliary 
                    xf = tf.placeholder(tf.float32, shape=(None,l))
                    l_id = tf.placeholder(tf.int32, shape=(None,))
                    l_id_len = tf.placeholder(tf.int32,shape=(1,))
                    positive_id = tf.placeholder(tf.int32, shape=(None,))
                    positive_len = tf.placeholder(tf.int32,shape=(1,))
                    r = tf.placeholder(tf.float32,shape=(None,))


                    image_i = tf.placeholder(tf.float32, shape=(1,l))
                    image_j = tf.placeholder(tf.float32, shape=(1,l))

                    with tf.variable_scope("item_level"):
                        user_latent = tf.get_variable("user_latent", [n, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3))
                        item_latent = tf.get_variable("item_latent", [m, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3)) 
                        aux_item = tf.get_variable("aux_item", [m, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3))
                        Wu = tf.get_variable("Wu", [n,m,k],  
                                                              initializer=tf.contrib.layers.xavier_initializer())
                        Wy = tf.get_variable("Wy", [n,m,k],   
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wa = tf.get_variable("Wa", [n,m,k],  
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wv = tf.get_variable("Wv", [n,m,embedding_dims],  
                                                            initializer=tf.contrib.layers.xavier_initializer())
                        #Wve =  tf.get_variable("Wve", [embedding_dims,l],  
                        #                                    initializer=tf.contrib.layers.xavier_initializer())

                        #每個user 對於每個YouTuber都有一個權重
                        #w1拿掉，wu
                        #hyper?

                        aux_new = tf.get_variable("aux_new", [1,k], initializer=tf.constant_initializer(0.0))
                        ########## Error part, how to get auxisize dynamically
                        ####aux_size= tf.get_variable(name='aux_size', initializer=l_id.get_shape().as_list()[-1])

                    with tf.variable_scope('feature_level'):
                        embedding = tf.get_variable("embedding", [embedding_dims,l],
                                    initializer=tf.contrib.layers.xavier_initializer())
                        Beta = tf.get_variable("beta", [n,embedding_dims],
                                    initializer=tf.random_normal_initializer(0.01,0.001,seed=10))

                    #lookup the latent factors by user and id
                    u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
                    vi = tf.nn.embedding_lookup(item_latent, i) 
                    vj = tf.nn.embedding_lookup(item_latent, j)


                    wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user)) #(m*k)
                    wy = tf.squeeze(tf.nn.embedding_lookup(Wy, user)) #(m*k)
                    wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user)) #(m*k)
                    wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user)) #(m,l)
                    beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor


                    a_list=tf.Variable([])
                    q = tf.constant(0)
                    def att_cond(q,a_list):
                        return tf.less(q,l_id_len[0])
                    def att_body(q,a_list):
                        xfi = tf.expand_dims(xf[q],0) #(1,l)
                        wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        a_list = tf.concat([a_list,[(tf.nn.relu( tf.matmul(wuui, u, transpose_b=True) +
                                tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
                                tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
                                tf.matmul(wvui,tf.matmul(embedding,xfi, transpose_b=True)))[0][0])*r[q]]],0)
                        q += 1
                        return q,  a_list

                    _, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])

                    # for while for smoothing
                    #a_list_soft=tf.nn.softmax(a_list)
                    a_list_smooth = tf.add(a_list,0.0000000001)
                    a_list_soft = tf.divide(a_list_smooth,tf.reduce_sum(a_list_smooth, 0)) #without softmax

                    norm_par = [wu,wy,wa,wv]

                    wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[-1]),0)
                    wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[-1]),0)
                    waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[-1]),0)
                    wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[-1]),0)
                    wu_be_relu = tf.matmul(wuui, u, transpose_b=True)
                    wy_be_relu = tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[-1]),0), transpose_b=True)
                    wa_be_relu = tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[-1]),0), transpose_b=True)
                    wv_be_relu = tf.matmul(wvui, tf.matmul(embedding,tf.expand_dims(xf[-1],0), transpose_b=True))
                    last_be_relu = [wu_be_relu,wy_be_relu,wa_be_relu,wv_be_relu]

                    aux_np = tf.expand_dims(tf.zeros(k),0) #dimension (1,32)
                    q = tf.constant(0)
                    def sum_att_cond(q,aux_np):
                        return tf.less(q,l_id_len[0])

                    def sum_att_body(q,aux_np):
                        #aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                        aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)]) 
                        q += 1
                        return q, aux_np

                    _,aux_np = tf.while_loop(sum_att_cond,sum_att_body,[q,aux_np])

                    """
                    for q in range(3): #取q個auxliary item
                        aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                    """

                    aux_part = tf.matmul(aux_np, vi, transpose_b=True)
                    #tf.print('aux attention:',aux_np)
                    aux_np+=u #user_latent factor + sum (alpha*auxilary)
                    aux_new=tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np


                    latent_i_part = tf.matmul(aux_new, vi, transpose_b=True)
                    feature_i_part = tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                    latent_j_part = tf.matmul(aux_new, vj, transpose_b=True)
                    feature_j_part = tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))
                    only_aux_i_part = tf.matmul(aux_np, vi, transpose_b=True)
                    only_aux_j_part = tf.matmul(aux_np, vj, transpose_b=True)

                    #矩陣中對應函數各自相乘
                    # ex: tf.matmul(thetav,(tf.matmul(embedding, image_i, transpose_b=True)))
                    xui = tf.matmul(aux_new, vi, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                    xuj = tf.matmul(aux_new, vj, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))

                    xuij = tf.subtract(xui,xuj)


                    l2_norm = tf.add_n([
                                0.0001 * tf.reduce_sum(tf.multiply(u, u)),
                                0.0001 * tf.reduce_sum(tf.multiply(vi, vi)),
                                0.0001 * tf.reduce_sum(tf.multiply(vj, vj)),


                                paru_weight * tf.reduce_sum(tf.multiply(wu, wu)),
                                pary_weight * tf.reduce_sum(tf.multiply(wy, wy)),
                                pary_weight * tf.reduce_sum(tf.multiply(wa, wa)),
                                pary_weight * tf.reduce_sum(tf.multiply(wv,wv)),

                                beta_weight * tf.reduce_sum(tf.multiply(beta,beta)),
                                Embedding_weight * tf.reduce_sum(tf.multiply(embedding,embedding)),

                                ])

                    loss = l2_norm -tf.log(tf.sigmoid(xuij)) # objective funtion
                    train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss) #parameter optimize 
                    auc = tf.reduce_mean(tf.to_float(xuij > 0))

                    Ur, Yr, Ar, Er, Aur, Ayr, Aar, Avr,Br = training(single_feature+'1_Edims200')
                    print('Finish dims:,',embedding_dims)

In [ ]:
a = np.load('../Data/npy/Image_50_2048D.npy')
b = np.load('../Data/npy/Video_512D_6650.npy')
c = np.load('../Data/npy/Text_300D.npy')
d = np.load('../Data/npy/social_Norm15D.npy')
temp1 = np.concatenate((a, b), axis=1)
temp2=np.concatenate((c,d), axis=1)
all_3374 = np.concatenate((temp1,temp2), axis=1)

l =all_3374.shape[1]
#finish list:[image+video,image+text,video+text,social_text,social_image,social_video, following_video, following_social,following_image,following_text]
#finish list:[following_image_text,following_image_video,following_image_social,social_image_video,social_image_text,text_image_video,]
#following_text_video,following_text_social, following_video_social
#following,image,text,video,social
par_weights = [0.01]
beta_weights = [0.001]
Embedding_weights = [0.01]
#Embedding_dims = [150,160,170,180,190,210,220,230,240,250]
Embedding_dims = [200]
testcount = 0
finish_list = []
for paru_weight in par_weights:
    for pary_weight in par_weights:
        for beta_weight in beta_weights:
            for Embedding_weight in Embedding_weights:
                for embedding_dims in Embedding_dims:
                    clear_output()
                    print('Finished Dims',finish_list)
                    finish_list.append(embedding_dims)
                    print('Now Dims:',embedding_dims)
                    """
                    n: the number of users
                    m: the number of YouTubers
                    
                    k: latent dims
                    l: feature dims
                    """
                    tf.reset_default_graph()

                    user = tf.placeholder(tf.int32,shape=(1,))
                    i = tf.placeholder(tf.int32, shape=(1,))
                    j = tf.placeholder(tf.int32, shape=(1,))

                    #多少個auxliary 
                    xf = tf.placeholder(tf.float32, shape=(None,l))
                    l_id = tf.placeholder(tf.int32, shape=(None,))
                    l_id_len = tf.placeholder(tf.int32,shape=(1,))
                    positive_id = tf.placeholder(tf.int32, shape=(None,))
                    positive_len = tf.placeholder(tf.int32,shape=(1,))
                    r = tf.placeholder(tf.float32,shape=(None,))


                    image_i = tf.placeholder(tf.float32, shape=(1,l))
                    image_j = tf.placeholder(tf.float32, shape=(1,l))

                    with tf.variable_scope("item_level"):
                        user_latent = tf.get_variable("user_latent", [n, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3))
                        item_latent = tf.get_variable("item_latent", [m, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3)) 
                        aux_item = tf.get_variable("aux_item", [m, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3))
                        Wu = tf.get_variable("Wu", [n,m,k],  
                                                              initializer=tf.contrib.layers.xavier_initializer())
                        Wy = tf.get_variable("Wy", [n,m,k],   
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wa = tf.get_variable("Wa", [n,m,k],  
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wv = tf.get_variable("Wv", [n,m,embedding_dims],  
                                                            initializer=tf.contrib.layers.xavier_initializer())
                        #Wve =  tf.get_variable("Wve", [embedding_dims,l],  
                        #                                    initializer=tf.contrib.layers.xavier_initializer())

                        #每個user 對於每個YouTuber都有一個權重
                        #w1拿掉，wu
                        #hyper?

                        aux_new = tf.get_variable("aux_new", [1,k], initializer=tf.constant_initializer(0.0))
                        ########## Error part, how to get auxisize dynamically
                        ####aux_size= tf.get_variable(name='aux_size', initializer=l_id.get_shape().as_list()[-1])

                    with tf.variable_scope('feature_level'):
                        embedding = tf.get_variable("embedding", [embedding_dims,l],
                                    initializer=tf.contrib.layers.xavier_initializer())
                        Beta = tf.get_variable("beta", [n,embedding_dims],
                                    initializer=tf.random_normal_initializer(0.01,0.001,seed=10))

                    #lookup the latent factors by user and id
                    u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
                    vi = tf.nn.embedding_lookup(item_latent, i) 
                    vj = tf.nn.embedding_lookup(item_latent, j)


                    wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user)) #(m*k)
                    wy = tf.squeeze(tf.nn.embedding_lookup(Wy, user)) #(m*k)
                    wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user)) #(m*k)
                    wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user)) #(m,l)
                    beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor


                    a_list=tf.Variable([])
                    q = tf.constant(0)
                    def att_cond(q,a_list):
                        return tf.less(q,l_id_len[0])
                    def att_body(q,a_list):
                        xfi = tf.expand_dims(xf[q],0) #(1,l)
                        wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        a_list = tf.concat([a_list,[(tf.nn.relu( tf.matmul(wuui, u, transpose_b=True) +
                                tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
                                tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
                                tf.matmul(wvui,tf.matmul(embedding,xfi, transpose_b=True)))[0][0])*r[q]]],0)
                        q += 1
                        return q,  a_list

                    _, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])

                    # for while for smoothing
                    #a_list_soft=tf.nn.softmax(a_list)
                    a_list_smooth = tf.add(a_list,0.0000000001)
                    a_list_soft = tf.divide(a_list_smooth,tf.reduce_sum(a_list_smooth, 0)) #without softmax

                    norm_par = [wu,wy,wa,wv]

                    wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[-1]),0)
                    wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[-1]),0)
                    waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[-1]),0)
                    wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[-1]),0)
                    wu_be_relu = tf.matmul(wuui, u, transpose_b=True)
                    wy_be_relu = tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[-1]),0), transpose_b=True)
                    wa_be_relu = tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[-1]),0), transpose_b=True)
                    wv_be_relu = tf.matmul(wvui, tf.matmul(embedding,tf.expand_dims(xf[-1],0), transpose_b=True))
                    last_be_relu = [wu_be_relu,wy_be_relu,wa_be_relu,wv_be_relu]

                    aux_np = tf.expand_dims(tf.zeros(k),0) #dimension (1,32)
                    q = tf.constant(0)
                    def sum_att_cond(q,aux_np):
                        return tf.less(q,l_id_len[0])

                    def sum_att_body(q,aux_np):
                        #aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                        aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)]) 
                        q += 1
                        return q, aux_np

                    _,aux_np = tf.while_loop(sum_att_cond,sum_att_body,[q,aux_np])

                    """
                    for q in range(3): #取q個auxliary item
                        aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                    """

                    aux_part = tf.matmul(aux_np, vi, transpose_b=True)
                    #tf.print('aux attention:',aux_np)
                    aux_np+=u #user_latent factor + sum (alpha*auxilary)
                    aux_new=tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np


                    latent_i_part = tf.matmul(aux_new, vi, transpose_b=True)
                    feature_i_part = tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                    latent_j_part = tf.matmul(aux_new, vj, transpose_b=True)
                    feature_j_part = tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))
                    only_aux_i_part = tf.matmul(aux_np, vi, transpose_b=True)
                    only_aux_j_part = tf.matmul(aux_np, vj, transpose_b=True)

                    #矩陣中對應函數各自相乘
                    # ex: tf.matmul(thetav,(tf.matmul(embedding, image_i, transpose_b=True)))
                    xui = tf.matmul(aux_new, vi, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                    xuj = tf.matmul(aux_new, vj, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))

                    xuij = tf.subtract(xui,xuj)


                    l2_norm = tf.add_n([
                                0.0001 * tf.reduce_sum(tf.multiply(u, u)),
                                0.0001 * tf.reduce_sum(tf.multiply(vi, vi)),
                                0.0001 * tf.reduce_sum(tf.multiply(vj, vj)),


                                paru_weight * tf.reduce_sum(tf.multiply(wu, wu)),
                                pary_weight * tf.reduce_sum(tf.multiply(wy, wy)),
                                pary_weight * tf.reduce_sum(tf.multiply(wa, wa)),
                                pary_weight * tf.reduce_sum(tf.multiply(wv,wv)),

                                beta_weight * tf.reduce_sum(tf.multiply(beta,beta)),
                                Embedding_weight * tf.reduce_sum(tf.multiply(embedding,embedding)),

                                ])

                    loss = l2_norm -tf.log(tf.sigmoid(xuij)) # objective funtion
                    train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss) #parameter optimize 
                    auc = tf.reduce_mean(tf.to_float(xuij > 0))

                    Ur, Yr, Ar, Er, Aur, Ayr, Aar, Avr,Br = training('Image_Video_Text_Social_Edims200')
                    print('Finish dims:,',embedding_dims)

In [ ]:
def testing(U, Y, A, E, A1, Au, Ay, Aa, Av,B):
    #print(A1)
    result=np.zeros((test_amount,88))
    RS=np.zeros((test_amount,88))
    #test_idx --> Test 的 index
    max1 = 0
    maxu = 0
    maxy = 0
    maxa = 0
    maxv = 0
    min1 = 100000000000000000
    minu = 100000000000000000
    miny = 100000000000000000
    mina = 100000000000000000
    minv = 100000000000000000
    test_yes_id=[]
    for s in range(test_amount):
        print(s,test_idx[s])

        yes=[]
        sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
        #sample=result_yes_id[now]
        test_yes_id.append(sample)
        alpha=np.zeros([len(sample)])

        for a in range(len(sample)):
            r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
            #print(test_idx[s])
            #print(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0)))

            #Observe each part in attention , below par are all (128,1)
            testW1 = np.sum(A1[test_idx[s]])
            #print(A1[test_idx[s]])
            WuUu = np.sum(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T))
            WyYy = np.sum(np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T))
            WaAa = np.sum(np.dot(Aa[test_idx[s]],np.expand_dims(A[sample[a]],0).T))
            WvVy = np.sum(np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T))
            print('The sum of each par -->\nw1:',testW1,'\nWuU:',WuUu,'\nwyY:',WyYy,'\nWaA:',WaAa,'\nWvV:',WvVy)
            if testW1 > max1:
                max1 = testW1
            if testW1 < min1:
                min1 = testW1
            if WuUu > maxu:
                maxu = WuUu
            if WuUu < minu:
                minu = WuUu
            if WyYy > maxy:
                maxy = WyYy
            if WyYy < miny:
                miny = WyYy
            if WaAa > maxa:
                maxa = WaAa
            if WaAa < mina:
                mina = WaAa
            if WvVy > maxv:
                mxv = WvVy
            if WvVy < minv:
                minv = WvVy
            #Have w1
            #alpha[a]=np.dot(A1[test_idx[s]],(relu(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]],
            #                    np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T))))*r
            #Without w1
            alpha[a]=np.sum((relu(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]],
                                np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T))))*r
        mul=np.zeros((1,128))
        print('alpha------------',alpha)
        print('softmax alpha--------------',softmax(alpha))
        for i in range(len(sample)):
            mul+=softmax(alpha)[i]*A[sample[i]] #attention alpha*Ai part 
        new_mul=mul+U[test_idx[s]]  #(U+auxilary)
        for k in range(88):
            result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
            RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]],np.dot(E, all_3374[k].T))
    #print(max1,maxu,maxy,maxa,maxv)
    #print(min1,minu,miny,mina,minv)
    return RS

In [ ]:
import os 
#從grid_search_weight中找尋不同的file 
path = '../Data/grid_search_weight/0105/'
files = os.listdir(path)
#U, Y, A, E, A1, Au, Ay, Aa, Av,B =sess.run([user_latent, item_latent, aux_item, embedding, W1, Wu, Wy, Wa, Wv,Beta])
for file in files:
    par_data = np.load(path+file)
    U = par_data['U']
    Y = par_data['Y']
    A = par_data['A']
    E = par_data['E']
    W1 = par_data['W1']
    Wu = par_data['Wu']
    Wy = par_data['Wy']
    Wa = par_data['Wa']
    Wv = par_data['Wv']
    B = par_data['B']
    RS = testing(U, Y, A, E, W1, Wu, Wy, Wa, Wv,B)

# Get latent factor and Each weight

In [ ]:
U, Y, A, E, A1, Au, Ay, Aa, Av,B =sess.run([user_latent, item_latent, aux_item, embedding, W1, Wu, Wy, Wa, Wv,Beta])

In [ ]:
print('User latent shape: ',U.shape)
print('photo latent shape: ', Y.shape)
print('Auxilary latent shape: ',A.shape)
print('W1 weight shape: ',A1.shape)
print('Wu weight shape:',Au.shape)
print('Wy weight shape:', Ay.shape)
print('Wa weight shape:',Aa.shape)
print('Wv weight shape:',Av.shape)
print('Beta shape:',B.shape)
print('Embedding shape:',E.shape)

In [ ]:
U, Y, A, E, Au, Ay, Aa, Av,B =sess.run([user_latent, item_latent, aux_item, embedding, Wu, Wy, Wa, Wv,Beta])

# Testing Part

In [ ]:
result=np.zeros((test_amount,88))
RS=np.zeros((test_amount,88))
#test_idx --> Test 的 index

test_yes_id=[]
for s in range(test_amount):
    print(s,test_idx[s])

    yes=[]
    sample=random.sample(train_t[test_idx[s]],len(train_t[test_idx[s]])) #從training part 的positive feedback 取出YouTuber 當成Auxilary
    #sample=result_yes_id[now]
    test_yes_id.append(sample)
    alpha=np.zeros([len(sample)])
    
    for a in range(len(sample)):
        r =np.max(YouTuber_category[sample[a]]*user_category_norm[test_idx[s]]) #sample a 的category vec *user_category vec
        #print(test_idx[s])
        #print(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0)))
        
        #Observe each part in attention , below par are all (128,1)
        testW1 = np.sum(np.multiply(A1[test_idx[s]],A1[test_idx[s]]))
        WuUu = np.sum(np.multiply(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T),np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T)))
        WyYy = np.sum(np.multiply(np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T),np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)))
        WaAa = np.sum(np.multiply(np.dot(Aa[test_idx[s]],np.expand_dims(A[sample[a]],0).T),np.dot(Aa[test_idx[s]],np.expand_dims(A[sample[a]],0).T)))
        WvVy = np.sum(np.multiply(np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T),np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T)))
        print('w1:',testW1,'\nWuU:',WuUu,'\nwyY:',WyYy,'\nWaA:',WaAa,'\nWvV:',WvVy)
        
        alpha[a]=np.dot(A1[test_idx[s]],(relu(np.dot(Au[test_idx[s]],np.expand_dims(U[test_idx[s]],0).T)+np.dot(Ay[sample[a]],np.expand_dims(Y[sample[a]],0).T)+np.dot(Aa[test_idx[s]],
                            np.expand_dims(A[sample[a]],0).T)+ np.dot(Av[test_idx[s]],np.expand_dims(all_3374[sample[a]],0).T))))*r
    mul=np.zeros((1,128))
    print('alpha------------',alpha)
    print('softmax alpha--------------',softmax(alpha))
    for i in range(len(sample)):
        mul+=softmax(alpha)[i]*A[sample[i]] #attention alpha*Ai part 
    new_mul=mul+U[test_idx[s]]  #(U+auxilary)
    for k in range(88):
        result[s][k]=np.dot(new_mul,Y[k].T) #(U+auxilary)*photo latent factor
        RS[s][k] = np.dot(new_mul,Y[k].T)+np.dot(B[test_idx[s]],np.dot(E, all_3374[k].T))
#print(RS[s])

In [ ]:
#取出test的資料
testRS = np.zeros((test_amount,yt_test_amount)) #shape 150*20
target = np.zeros((test_amount,yt_test_amount))
#test_t 是true的
#test_f 是false的
        
for z in range(test_amount):
    user_id = test_idx[z]
    #positive target YouTuber list
    youtube_t = test_t[z] 
    #not target YouTuber list
    youtube_f = test_f[z]
    
    #前兩個放target的RS
    for i in range(len(youtube_t)):
        testRS[z][i] = RS[z][youtube_t[i]]
        target[z][i] = 1
    for i in range(len(youtube_f)):
        testRS[z][i+len(youtube_t)] = RS[z][youtube_f[i]]

In [ ]:
target

In [ ]:
testRS

In [ ]:
sumtarget = 0
for i in range(len(target)):
    #print(np.sum(target[i]))
    sumtarget += np.sum(target[i])
print('num of positive data in testing:',sumtarget)
print('total testing data:',test_amount*yt_test_amount)

In [ ]:
def topN(sortlist,n):
    topList = []
    for i in range(n):
        topList.append(sortlist.index(max(sortlist)))
        #print(max(sortlist))
        #print(sortlist.index(max(sortlist)))
        sortlist[sortlist.index(max(sortlist))] = -1000000000
    return topList

In [ ]:
count_0_all = []
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),int(np.sum(target[i])))
    count_0_all.append(top_0)
    print(top_0)

acc_0 = 0
total = 0
for i in range(len(count_0_all)):
    for j in range(len(count_0_all[i])):
        #print(int(np.sum(target[i])))
        total+=int(np.sum(target[i]))
        if count_0_all[i][j] < int(np.sum(target[i])): #代表是0或1 (也就是target)
            acc_0 += 1
avg_acc = acc_0/100
print('avg_accuarcy for count_0:',avg_acc)
print(acc_0)

In [ ]:
acc_0

In [ ]:
total

# Top 1 

In [ ]:
def F1_score(prec,rec):
    f1 = (2*prec*rec)/(prec+rec)
    return f1

In [ ]:
correct = 0
for i in range(len(testRS)):
    top_0 = topN(list(testRS[i]),1) #取一個
    count_0_all.append(top_0)
    print(top_0)
    if top_0[0] < int(np.sum(target[i])):
        correct += 1

In [ ]:
top1_prec = correct/len(testRS)
top1_recall = correct/(sumtarget)
print('prec ',top1_prec,'recall ',top1_recall)

In [ ]:
#f1 score
print('F1_score:',F1_score(top1_prec,top1_recall))

# Top 3

In [ ]:
correct = 0
for i in range(len(testRS)):
    top_3 = topN(list(testRS[i]),3) #取一個
    count_0_all.append(top_3)
    #print(top_3)
    for j in range(len(top_3)):
        if top_3[j] < int(np.sum(target[i])):
            correct += 1

In [ ]:
top3_prec = correct/(len(testRS)*3)
top3_recall = correct/(sumtarget)
print('prec ',top3_prec,'recall ',top3_recall)

In [ ]:
#f1 score
print('F1_score:',F1_score(top3_prec,top3_recall))

# Top 5

In [ ]:
correct = 0
for i in range(len(testRS)):
    top_5 = topN(list(testRS[i]),5) #取一個
    count_0_all.append(top_5)
    #print(top_5)
    for j in range(len(top_5)):
        if top_5[j] < int(np.sum(target[i])):
            correct += 1

In [ ]:
top5_prec = correct/(len(testRS)*5)
top5_recall = correct/(sumtarget)
print('prec ',top5_prec,'recall ',top5_recall)

In [ ]:
#f1 score
print('F1_score:',F1_score(top5_prec,top5_recall))

In [ ]:
#User_latent_factor = loaddata['User']
#YouTuber_latent_factor = loaddata['YouTuber']

In [ ]:
np.savez('../Data/latent_factor/YRM_up10_ALL/Final/1226.npz', User=U, YouTuber=Y)

In [ ]:
def softmax(x):
    x = x - np.max(x)
    exp_x = np.exp(x)
    softmax_x = exp_x / np.sum(exp_x)
    return softmax_x

In [ ]:
softmax([-0.000000000000000000000000000000000000001,0.00000000000000000000000000000000000001])

In [ ]:
C = tf.constant([1.1,2.7,6.1,4.7,5.5,6.5])
B = tf.divide(C,tf.reduce_sum(C))

In [ ]:
B

In [ ]:
tryy = np.array([1,2,3,4,5])
np.add(tryy,1)

In [2]:
import numpy as np
np.prod((2,3))

6